In [ ]:
# Installing Detectron2 and the dependencies(PyTorch)

!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
!pip install cython pyyaml==5.1
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-agzxd8sq
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-agzxd8sq
     |████████████████████████████████| 276kB 8.0MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1-cp36-none-any.whl size=44898 sha256=f8f63381a352f1cddc7e362e39d3730cfb8ec56a71fa46edd211a97f7d4c6dc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-4zor4bjr/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=c03727b2d0a98f978f03cd74ad68ed69ec1d043cb1545b195ac9a208dca6f44e
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully

In [ ]:
# Setup Detectron 2

%cd detectron2_repo
!python setup.py install

/content/detectron2_repo
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running install
running bdist_egg
running egg_info
writing detectron2.egg-info/PKG-INFO
writing dependency_links to detectron2.egg-info/dependency_links.txt
writing requirements to detectron2.egg-info/requires.txt
writing top-level names to detectron2.egg-info/top_level.txt
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:304: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'detectron2.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying detectron2/__init__.py -> build/lib.linux-x86_64-3.6/detectron2
creating build/lib.linux-x86_64-3.6/detectron2/model_zoo
copying detectron2/model_zoo/model_zoo.py -> build/lib.linux-x86_64-3.6/detectron2/mode

In [ ]:
import detectron2
print(detectron2.__version__)

0.1.3


In [ ]:
%cd ..
import os
os.listdir()

/content


['.config', 'detectron2_repo', 'sample_data']

In [ ]:
import os
import numpy as np
import json
from torch._C import *
import itertools
import cv2
from detectron2 import _C
from detectron2.structures import BoxMode
from google.colab import drive
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
drive.mount('/gdrive')

In [ ]:
%cd ..
os.listdir()

In [ ]:
def get_invoice_dicts(img_dir): 

  # Getting the data 
  json_file = os.path.join('gdrive/My Drive/Colab Notebooks/', "labels.json")
  with open(json_file) as f:
    imgs_anns = json.load(f)
 
  dataset_dicts = []
  for k, v in imgs_anns.items():
    record = {}
          
    filename = os.path.join(img_dir, v["filename"])
    height, width = cv2.imread(filename).shape[:2]
    record["filename"] = imgs_anns[str(k)]['filename']
    record["height"] = height
    record["width"] = width
        
    annos = v["regions"]
    objs = []
    
    for k, anno in annos.items():
      anno = anno["shape_attributes"]
      px = anno["all_points_x"]
      py = anno["all_points_y"]
      poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
      poly = list(itertools.chain.from_iterable(poly))
      obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": 0,
              "iscrowd": 0
              }
      objs.append(obj)
    
    record["annotations"] = objs
    dataset_dicts.append(record)
  return dataset_dicts

In [ ]:
register_coco_instances('Invoices',{},'gdrive/My Drive/Colab Notebooks/labels.json','gdrive/My Drive/Colab Notebooks/images')


In [ ]:
MetadataCatalog.get("Invoices").set(thing_classes=["Tables"])
invoice_metadata = MetadataCatalog.get("Invoices")

In [ ]:
invoice_metadata

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("content/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("Invoices",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
import torch 
print(torch.__version__)

print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
print(torch.cuda.current_device())